In [ ]:
from flow_matching_jax.flow_matching_sampling import get_sde as get_sde_fm
from flow_matching_jax.flow_matching_sampling import SDE as SDE_FM
from flow_matching_jax.models.get_model import TrainState
from flow_matching_jax.train import restore_checkpoint, create_train_state
from sdes.sdes import SDE
import jax
from jax import random
import jax.numpy as jnp
import orbax


def restore_checkpoint_from_config_directory(config, workdir):
    rng = jax.random.PRNGKey(config.seed)
    state = create_train_state(rng, config)
    orbax_checkpointer = orbax.checkpoint.PyTreeCheckpointer()
    options = orbax.checkpoint.CheckpointManagerOptions(max_to_keep=2, create=True)
    checkpoint_manager = orbax.checkpoint.CheckpointManager(
        workdir, orbax_checkpointer, options)
    state = restore_checkpoint(state, checkpoint_manager)
    return config, state


def get_sde(config, workdir):
    config, state = restore_checkpoint_from_config_directory(config, workdir)
    sigma = lambda t: 0.5
    sde_fm = get_sde_fm(state, sigma, True)

    drift = sde_fm.drift
    sigma = lambda t, x, dBt: dBt * sigma(t)
    covariance = lambda t, x, dBt: dBt * sigma(t)**2
    sigma_transpose_inv = lambda t, x, dBt: dBt / sigma(t)

    sde = SDE(drift, sigma, covariance, sigma_transpose_inv)
    


    y_obs = -1
    y_obs_arr = jnp.ones(D) * y_obs
    sde, control = ou_sde(alpha)
    control = true_control(sde, y_obs)

    sde = vmap_sde_dimension(sde)
    control = vmap_control_only_first_dimension(control, D)

    y_initial_validation = jnp.ones(D) * y_obs
    y_initial_validation = y_initial_validation.at[0].set(1)

    return sde, control, y_obs_arr, y_initial_validation, f"OU Problem ({alpha})"


2025-03-26 10:14:09.816224: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742984049.825090   42928 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742984049.829263   42928 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.3.0) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_min

In [2]:
from flow_matching_jax.configs.fashion_mnist import get_config

config = get_config()
config, state = restore_checkpoint_from_config_directory(config, "flow_matching_jax/workdir/fashion_mnist")

initializing


ValueError: Checkpoint path should be absolute. Got flow_matching_jax/workdir/fashion_mnist/10500/default

data:
  batch_size: 1024
  cache: false
  channels: 1
  dataset: fashion_mnist
  eval:
    n_samples: 16
    timesteps: 1000
  image_size: 28
  origin: online
  shape: !!python/tuple
  - 28
  - 28
  - 1
ema:
  beta: 0.995
  inv_gamma: 1.0
  min_value: 0.0
  power: 0.6666666666666666
  update_after_step: 100
  update_every: 10
fm:
  targets: direction
model:
  dim: 64
  dim_mults: !!python/tuple
  - 1
  - 2
  - 4
  is_batched: true
  name: unet
optim:
  beta1: 0.9
  beta2: 0.99
  eps: 1.0e-08
  lr: 0.001
  optimizer: Adam
seed: 1995
training:
  grad_clip: 1.0
  half_precision: false
  log_every_steps: 100
  num_train_steps: 20000
  save_and_sample_every: 500
wandb:
  entity: null
  job_type: training
  log_model: false
  log_sample: true
  log_train: true
  name: null
  project: fashion_mnist